In [71]:
import csv
import xml.etree.ElementTree as ET
from pprint import pprint
from pathlib import Path
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [70]:
data = Path("sample")
files = list(data.glob("*.xml"))

In [122]:
persons_columns = ['ID', 'Fullname', 'Nationality', 'CountryOfBirth', 'CityOfBirth']
persons = pd.DataFrame(columns=persons_columns)

pub_columns = ['personID', 'DOI', 'title', 'year', 'journal', 'ISSN']
publications = pd.DataFrame(columns=pub_columns)

degree_columns = ['personID', 'tag', 'started', 'finished', 'code1', 'code2', 'type_of_phd', 'institute']
degrees = pd.DataFrame(columns=degree_columns)

In [ ]:
for f in tqdm(files):
    try:
        root = ET.parse(str(f)).getroot()
    except:
        print("Couldn't read '{}' | Skipping".format(f.name))
        continue
    
    # General data
    data_node = root.find("DADOS-GERAIS")
    
    person_id = root.get('NUMERO-IDENTIFICADOR')
    full_name = data_node.get('NOME-COMPLETO')
    city_of_birth = data_node.get('CIDADE-NASCIMENTO')
    country_of_birth = data_node.get('PAIS-DE-NASCIMENTO')
    nationality = data_node.get('PAIS-DE-NACIONALIDADE')
    
    row = [person_id, full_name, nationality, country_of_birth, city_of_birth]
    persons.loc[len(persons)+1] = row
    
    # Education
    edu_node = data_node.find("FORMACAO-ACADEMICA-TITULACAO")

    for degree_tag in ["DOUTORADO", "POS-DOUTORADO"]:
        node = edu_node.find(degree_tag)
        
        if not node:
            continue

        inst = node.get("NOME-INSTITUICAO")
        started = node.get("ANO-DE-INICIO")
        finished = node.get("ANO-DE-CONCLUSAO")
        code_1 = node.get("CODIGO-INSTITUICAO-SANDUICHE")
        code_2 = node.get("CODIGO-INSTITUICAO-CO-TUTELA")
        type_of_phd = node.get("TIPO-DOUTORADO")

        row = [person_id, degree_tag, started, finished, code_1, code_2, type_of_phd, inst]
        degrees.loc[len(degrees)+1] = row
            
    # Publications
    articles = root.find("PRODUCAO-BIBLIOGRAFICA/ARTIGOS-PUBLICADOS")
    
    if not articles:
        continue
        
    for article in articles:
        basics = article.find("DADOS-BASICOS-DO-ARTIGO")
        details = article.find("DETALHAMENTO-DO-ARTIGO")
        
        doi = basics.find("DOI")
        year = basics.get("ANO-DO-ARTIGO")
        title = basics.get("TITULO-DO-ARTIGO")
        
        journal = details.get("TITULO-DO-PERIODICO-OU-REVISTA")
        issn = details.get("ISSN")
        
        row = [person_id, doi, year, title, journal, issn]
        publications.loc[len(publications)+1] = row

Couldn't read '8845087913178096.xml' | Skipping


In [124]:
persons.sample(10)

,ID,Fullname,Nationality,CountryOfBirth,CityOfBirth
54,7012348447122522,Eunice Maia de Andrade,Brasil,Brasil,Limoeiro do Norte
82,9883445785268500,Giovana Xavier da Conceição Nascimento,Brasil,Brasil,Rio de Janeiro
77,7563136807559182,Ricardo Luiz Sapia de Campos,Brasil,Brasil,São Caetano do Sul
8,9128090974948413,Marilise Escobar Burger,Brasil,Brasil,SANTA MARIA
86,3715055413703709,Mônica Vasconcellos,Brasil,Brasil,Rio de Janeiro
2,,Gilberto Montibeller Neto,None,Brasil,Florianópolis
72,5867341426597609,Thiago Oliveira da Motta Sampaio,Brasil,Brasil,Rio de Janeiro
94,8080838290715777,Ana Cecília Ribeiro Cruz,Brasil,Brasil,BELEM
53,1641673656667170,Tiago Pereira do Nascimento,Brasil,Brasil,Salvador
48,8496716957724133,Kepler de Souza Oliveira Filho,Brasil,Brasil,Salvador


In [139]:
degrees.sample(10)

,personID,tag,started,finished,code1,code2,type_of_phd,institute
22,4362051219348099,DOUTORADO,1993,1998,,,N,Universidade de São Paulo
61,8496716957724133,POS-DOUTORADO,1990,1991,None,None,None,Université de Montreal
13,1626484708254412,DOUTORADO,2002,2006,,,N,Universidade Federal do Ceará
16,4803537041404612,DOUTORADO,2008,2011,,,N,Universidade Federal de Lavras
25,0957394759457048,POS-DOUTORADO,2011,,None,None,None,Universidade de São Paulo
48,2281256929844028,DOUTORADO,2006,2010,,,N,Universidade de Brasília
42,8378701346846058,DOUTORADO,2004,2009,,,S,Universidade Estadual de Campinas
65,0412836563259361,DOUTORADO,2005,2009,,,S,Pontifícia Universidade Católica do Rio Grande...
26,8365078023155571,DOUTORADO,1989,1995,,,,Freie Universität Berlin
57,2306964700488382,DOUTORADO,1977,1983,,,,Instituto Nacional de Pesquisas Espaciais


In [149]:
publications.sample(10)

,personID,DOI,title,year,journal,ISSN
3670,6202668326517008,None,2002,Bioatividade de ácidos húmicos: efeitos sobre ...,Pesquisa Agropecuária Brasileira,0100204X
2990,8929162910172931,None,2009,Controlled fabrication of gold nanoparticles b...,"Materials Science & Engineering. C, Biomimetic...",09284931
1154,8128743330371501,None,2007,"O papel do aleitamento materno, da dieta e do ...",Jornal Brasileiro de Pneumologia (Impresso),18063713
792,4362051219348099,None,2011,SMARCA5 Methylation and Expression in Gastric ...,Cancer Investigation,07357907
2217,8496716957724133,None,2017,Evidence from for Rapid Rotation in the Descen...,The Astrophysical Journal,20418213
1417,2162607115118731,None,2003,Da compensação de tributos administrados pela ...,Revista Dialética de Direito Tributário,14137097
3248,9319381761309495,None,2000,Infecção puerperal,GO Atual (Rio de Janeiro),14143534
523,4803537041404612,None,2007,Níveis de saturação por bases para a produção ...,Revista Brasileira de Plantas Medicinais,15160572
3067,4859954582615304,None,2007,Light-Driven Horseradish Peroxidase Cycle by U...,Photochemistry and Photobiology,00318655
2841,6190787636978029,None,2013,"Pacto Internacional sobre Direitos Econômicos,...",Revista do Instituto do Direito Brasileiro,21827567


In [ ]:
# Export